In [ ]:
### Run this cell if running script in Google Colaboratory
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
### NOTE: If running script on operating system for first time, 
### run this cell.
!pip install tensorflow

In [5]:
### Required libraries for script
import numpy as np
import pandas as pd
from keras.models import load_model  ### This library requires tensorflow to be installed on your OS (automatically installed in Google Colaboratory).
import os


#### Values for transformation and unstransformation of input and output data ####

raw_min_x = [0.630073205118252,1.43810276557797,2.96269241947579,1.76472729118331, 
           3.44590962334884,5.1375514408174,0.000338858580945619,4.00369019395397,
           0,0,10.0148352591427,6.62936325343745]

raw_max_x = [1.83601494659418,3.73669154870064,7.87105554302205,12.5367912114883,
           3.91209676270801,9.52841555145308,0.0989619485186843,4.53259949315326,
           18.5541124009785,12.0109345668791,16.3542110492893,17.2997495062243]

#################################################################################



##### Defined functions for data transformation and scaling #####

def transform(value):  
  """ transforms the value or data provided to a natural logarithmic form """
  variables = []
  for vals in value:
    trans = np.sign(vals)*(np.log(abs(vals)+1))
    variables.append(trans)
  return variables

def minmax_norm(value, raw_min, raw_max):
  """ applies the linear min-max normalization """
  variables = []
  for i in range(0,len(raw_min)):
    norm = (0.7 * ((value[i]-raw_min[i])/(raw_max[i]-raw_min[i]))) + 0.15
    variables.append(norm)
  return variables 

def untransform_cap_loss(value):
  """ untransforms the value or data provided to its original form """
  untrans = np.exp(((value-0.15)/0.7)*(14.1042777914158-0.693147180559945)+0.693147180559945)-1  
  return untrans

###################################################################


### Loading Model --------  On line below, be sure to change and set the 'path' in load_model('path') for the file path of the 'calibrated_best_model.hdf5' file on your OS or Google Drive
model = load_model(r'C:\Users\coxal\OneDrive - Saint Louis University\RSI (I)\7 Documents and Presentations\Final Report\Final Draft\Scripts\calibrated_best_model.hdf5')


### Input Values ###

mean_mnth_precip = float(input('Mean Monthly Precipitation (in/mo.): ')) 
max_mnth_precip = float(input('Max Monthly Precipitation (in): '))
cumulative_precip = float(input('Cumulative Precip. (in): ')) 
basin_area = float(input('Basin Area (mi\N{SUPERSCRIPT TWO}): '))
avg_basin_lat = float(input('Avg. Basin Lat. (°): '))
basin_rlf = float(input('Basin Relief (ft): '))   
channel_slp = float(input('Channel Slope: '))    
curve_number = float(input('Curve Number: '))  
tot_upstr_nrm_str = float(input('Total Upstream Normal Storage (acre-ft): ')) 
tot_upstrm_dam_ht = float(input('Total Upstream Dam Height (ft): ')) 
hydraulic_len = float(input('Hydraulic Length (ft): '))     
initial_capacity =  float(input('Initial Capacity (acre-ft): '))     

#####################


#Create list of input variables in specified order --- NOTE: Do not change order. Order of variables important for accurate model performance.
predictor_variables = [mean_mnth_precip, max_mnth_precip, cumulative_precip, basin_area,  
                       avg_basin_lat,basin_rlf, channel_slp, curve_number, tot_upstr_nrm_str,
                       tot_upstrm_dam_ht, hydraulic_len, initial_capacity]

ls_values = minmax_norm(transform(predictor_variables),raw_min_x,raw_max_x)  #log transformation and minmax scaling of input values
ls_array = np.array(ls_values).reshape(1,-1)  #create numpy array of input values for prediction 
prediction = model.predict(ls_array)  #performs prediction based on input values
predicted_value = untransform_cap_loss(prediction)  #untransforms the resulting predicted value
print('*'*53)
print('Predicted Capacity Loss Value (acre-ft): ', predicted_value[0][0])  #displays predicted result

Mean Monthly Precipitation (in/mo.): 1.348
Max Monthly Precipitation (in): 5.529
Cumulative Precip. (in): 250.4
Basin Area (mi²): 262227
Avg. Basin Lat. (°): 46.09
Basin Relief (ft): 12504
Channel Slope: 0.001035
Curve Number: 76.33
Total Upstream Normal Storage (acre-ft): 108849468
Total Upstream Dam Height (ft): 147002
Hydraulic Length (ft): 12086074
Initial Capacity (acre-ft): 6281079
1/1 [==============================] - 0s 253ms/step
*****************************************************
Predicted Capacity Loss Value (acre-ft):  132644.69
